<a href="https://colab.research.google.com/github/2203A52082/2082_Deeplearning/blob/main/2203A52082_Week_7_A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "/content/Load Data in kW.xlsx"
df = pd.read_excel(file_path, sheet_name=0)

def preprocess_data(df):
    df.columns = ['Load']  # Naming the column
    df['Prev_Week_Load'] = df['Load'].shift(168)  # Assuming hourly data with 168-hour (7-day) shift
    df.dropna(inplace=True)  # Remove rows with NaN values
    return df

df = preprocess_data(df)

# Normalize the data
def normalize_data(df):
    df = (df - df.min()) / (df.max() - df.min())
    return df

df = normalize_data(df)

# Split data into train and test (90:10)
train_size = int(0.9 * len(df))
train_data = df[:train_size]
test_data = df[train_size:]

# Extract features and targets
X_train = train_data['Prev_Week_Load'].values
y_train = train_data['Load'].values
X_test = test_data['Prev_Week_Load'].values
y_test = test_data['Load'].values

# Initialize parameters
m, c = 1, -1  # Given initial values
eta = 0.1  # Learning rate
epochs = 100  # Number of epochs
gm_sq, gc_sq = 0, 0  # Accumulators for AdaGrad

def compute_gradients(X, y, m, c):
    error = y - (m * X + c)
    gm = -np.mean(error * X)
    gc = -np.mean(error)
    return gm, gc

# Training using AdaGrad optimizer
for epoch in range(epochs):
    gm, gc = compute_gradients(X_train, y_train, m, c)
    gm_sq += gm ** 2
    gc_sq += gc ** 2
    m -= (eta / (np.sqrt(gm_sq) + 1e-8)) * gm
    c -= (eta / (np.sqrt(gc_sq) + 1e-8)) * gc

# Prediction function
def predict(X, m, c):
    return m * X + c

# Compute error metrics
def compute_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    return mse, rmse, mae

# Evaluate on train data
y_train_pred = predict(X_train, m, c)
mse_train, rmse_train, mae_train = compute_metrics(y_train, y_train_pred)

# Evaluate on test data
y_test_pred = predict(X_test, m, c)
mse_test, rmse_test, mae_test = compute_metrics(y_test, y_test_pred)

# Print metrics
print("Training Data: MSE =", mse_train, "RMSE =", rmse_train, "MAE =", mae_train)
print("Test Data: MSE =", mse_test, "RMSE =", rmse_test, "MAE =", mae_test)

Training Data: MSE = 0.002212639630020053 RMSE = 0.047038703532517274 MAE = 0.02910035309907448
Test Data: MSE = 0.0007375332084534883 RMSE = 0.027157562638305526 MAE = 0.027133057921442383
